## Generate H3 index by Country Boundary

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import h3

# Load country boundary
gdf = gpd.read_parquet(r"..\data\boundaries\India.parquet")

# Ensure geometry is in WGS84 (required by H3)
gdf = gdf.to_crs(epsg=4326)
gdf = gdf[gdf['Name']=='India'].copy()
geometry = gdf.geometry.union_all()

In [ ]:
resolution = 6
cells = polygon_to_h3(geometry, resolution)

def h3_cell_to_polygon(h):
    # H3 returns (lat, lon)
    boundary_latlon = h3.cell_to_boundary(h)

    # ✅ Convert to (lon, lat) for Shapely
    boundary_lonlat = [(lon, lat) for lat, lon in boundary_latlon]

    return Polygon(boundary_lonlat)

df = pd.DataFrame({"h3_id": list(cells)})

df["geometry"] = df["h3_id"].apply(
    lambda h: h3_cell_to_polygon(h)
)

gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

gdf.to_parquet(rf"..\data\boundaries\India_h3_res{resolution}.parquet")

In [ ]:
gdf.shape

## Generate h3 cell boundaries using `duckdb`

In [ ]:
import duckdb


con = duckdb.connect()

con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL spatial; LOAD spatial;")
con.execute("INSTALL h3 FROM community;")
con.execute("LOAD h3;")

india_parquet = r"..\data\boundaries\India.parquet"

resolution = 7

query = f"""
WITH geom AS (
  SELECT ST_Union_Agg(ST_MakeValid(geometry)) AS g
  FROM read_parquet('{india_parquet}')
  WHERE Name = 'India'
),
parts AS (
  SELECT (d.unnest).geom AS geom
  FROM geom,
  UNNEST(ST_Dump(g)) AS d(unnest)
),
cells AS (
  SELECT UNNEST(h3_polygon_wkt_to_cells(ST_AsText(geom), {resolution})) AS h3_id
  FROM parts
)
SELECT
  h3_id,
  ST_GeomFromText(h3_cell_to_boundary_wkt(h3_id)) AS geom
FROM cells
"""

df = con.execute(query).df()

In [ ]:
df.head()